# Lab 1 : Spatial Autocorrelation

In this lab, we explore the measurement of spatial autocorrelation by Moran's index. 

We will use both _R_ in this notebook and a package called _GeoDa_ (installed on the lab computers) to do this. Each has advantages. We suggest you work in this notebook to the point where you begin to find the code involved a bit bewildering. It's OK if that happens: don't panic, just switch to _GeoDa_. 

Even if you don't get pushed past your tolerance for code, you are encouraged to also explore the application of these measures using _GeoDa_.

Whichever tool you use to complete the lab, you will find the submission instructions and questions at the end of this notebook.

## Getting started
First we have to import some packages for handling spatial data. We have used these before (in the Lab 0 notebook), except for `spdep` which implements various methods for exploring **spatial dependence** in data, one of which is Moran's _I_.

In [ ]:
library(spdep)
library(classInt)
library(rgdal)
library(RColorBrewer)

Now that's done, we can, move on to...

## Read in the data and take a look at it
Read the shape file from the data directory

In [ ]:
auck <- readOGR("data/ak-TB-ethnicity-0506.shp", integer64="allow.loss")

That means that the data are now all associated with an object called `auck`. We can see a summary:

In [ ]:
summary(auck)

This shows us that we have a number of data attributes, including `TB_CASES` and `TB_RATE` which record numbers of cases and rate (per 100,000 population) of tuberculosis in census area units (AUs) in Auckland, New Zealand.

Spatial data has a 'geography' component and a 'data attributes' component. We can look more closely at the data part by splitting out the data into an _R_ data frame, and using the `head`() command to see the first few rows.

In [ ]:
#The data frame -> row and header of data, in "tabular" format
#let's make df the dataframe
df <- data.frame(auck)
nrow(df)
head(df, n=10)

The data table is all very well, but since these are geographical data, what we really want to do is...

## Exploring the data in maps

As in the previous lab, we will make some choropleth maps to examine the various data of interest in this setting. To make this a bit less arduous, here is that simple choropleth mapping function from the previous notebook, which you can use to make maps of the different variables of interest. Make sure to run this cell, so that the `choro()` function that it defines is available in the rest of your session in this notebook.

In [ ]:
# Definition of a function to automate a series of commands and make a choropleth map
choro <- function(sf, varname, nclasses=5, pal='Reds', sty='equal', ttl=varname) {
    palette <- brewer.pal(nclasses, pal)
    classes <- classIntervals(sf[[varname]], nclasses, style=sty)
    colors <- findColours(classes, palette)
    plot(sf, col=colors, lwd=0.2)
    legend('top', ncol=3, legend=names(attr(colors, 'table')), fill=attr(colors, 'palette'), cex=0.8, bty='n')
    title(ttl)
}

Principally we want you to look at the tuberculosis rate (in cases per 100,000 population) `TB_RATE`,  and also at the different distributions of the four major New Zealand Census-defined ethnic groups, NZ European `EUR_P_06`, Māori `MAO_P_06`, Pasifika `PAC_P_06`, and Asian `ASI_P_06`. 

Use the above function in the cell below, to map the tuberculosis rate.

Don't forget that you have options for changing the map colors (`pal`), the number of classes (`nclasses`) and the classification method (`sty`) in this map and others you make.

In [ ]:
# Put a line of code here to map the tuberculosis rate data

Use the cell below to map all four of the major population groups. idea you might explore is to make all four of the ethnicity/race distribution maps in single display, by first issuing the `par(mfrow=c(2,2))` command, which will set up the display area for a 2 by 2 grid of maps. Then make four distinct maps.

In [ ]:
# this line sets up the graphic display for a two by two array of plots
# with narrower margins of 0.1 of the overall display area
par(mfrow=c(2,2), mai=rep(0.1,4))
# write a line of code to make a map
# write a line to make another map
# and another
# and then a fourth one

## Graphing the data
Keep in mind that you can also graph data, if desired.

In [ ]:
hist(auck$ASI_P_06, labels=T, main="Percent Asian (census-defined), 2006", xlab="Percent")

### Some background on Auckland's diverse population
It's worth recognizing the specificity of these groups to the New Zealand and particularly the Auckland setting. 

Māori are the population indigenous to New Zealand before European arrival in the 19th century. Pasifika reflect Auckland's place as 'the capital of Polynesia', with more people of Pacific Island heritage than anywhere in the world with the possible exception of Los Angeles (a metropolis 12 to 15 times more populous).  See for example, this footage of crowds of Aucklanders greeting the arrival of the Samoan (https://www.youtube.com/watch?v=NABfXFHumHg) and Tongan (https://www.youtube.com/watch?v=qyu646wHw08) rugby teams before the 2011 Rugby World Cup in Auckland.  Pasifika have been a substantial presence in Auckland since the 1950s when many arrived as 'guest workers'.  

More recently, Auckland, in common with Pacific Rim cities (including in California) has seen the rapid growth of large immigrant communities from across Asia, but especially East Asia, become a firmly established component of the city's greater than 40% foreign-born population.

As in the US, the 'Asian' population category is ridiculously broad, including South Asian (i.e. India-Pakistan), Chinese, and East Asian (Korea, Vietnam, etc.)  How race and ethnicity are defined in census terms is a vexed question, that varies from place to place.  For our present purpose, of note is that Māori and Pasifika populations in New Zealand are more socioeconomically challenged than the population in general, and that the Asian population are more likely to be recent immigrants than other segments of the population.

# Spatial autocorrelation

The focus of this lab is on measuring spatial autocorrelation. We are going to do this with the aim of identifying where the most dense clear clusters ('high-high') of high tuberculosis rates are to be found based on the data available. We will start with a simple autocorrelation analysis, and then proceed to the more complex business of local Moran's analysis, which is what we need to identify cluster locations.

The goal then is to relate the cluster locations, at least qualitatively to the different population distributions (mapped in the previous sections) and answer some questions that follow, at the end of the notebook.

We make no representation that the steps in this notebook (or using GeoDa) represent a thorough analysis of the topic at hand, but they do provide a sense of how getting a handle on spatial patterns can be generative of interesting perspectives and prompt further questions.

You will recall from lectures, that there are two aspects to measuring spatial autocorrelation, the first being 'nearness', so we will look at this first.

### Nearness - setting up the neighborhood structure
Here we will use the simplest possible definition of neighbors based on polygon contiguity. More elaborate concepts of 'nearness' can be explored using _GeoDa_, or if you are feeling brave, by delving into the [`spdep` package documentation](https://www.rdocumentation.org/packages/spdep/versions/0.7-4).

We make a neighbors object using the `poly2nb` function.

In [ ]:
# queen = False option means at least two boundary points must be 
# with the conventional name of a ‘rook’ relationship.
nb <- poly2nb(auck, row.names=auck$FIRST_CAU_, queen=FALSE)

We can get an idea of what is in the `nb` object by looking using the `summary()` function.

In [ ]:
summary(nb)

To make sure you have some idea of what is going on here, consider the following question:

**How many regions are there with no links?**

If you are unsure, ask!

We can also check what the neighborhoods look like, by plotting `nb` on top of a basemap.

In [ ]:
plot(auck, col='gray', border='white', lwd=0.35)
plot(nb, coordinates(auck), col='red', cex=0.5, lwd=0.5, add=TRUE)

## Moran's *I* in equation form

$I = \frac{n}{\sum_{i=1}^n (y_i - \bar{y})^2} \frac{\sum_{i=1}^n \sum_{j=1}^n w_{ij}(y_i - \bar{y})(y_j - \bar{y})}{\sum_{i=1}^n \sum_{j=1}^n w_{ij}}$

In [ ]:
# Let's first see what manual computation of Moran's I looks like.
# This part is optional, but feel free to uncomment the lines 
# needed to run the manual calculation

# 1. n is the number of observations (length of our dataset)
# n <- length(auck)

# 2. we set y to the column of PC_ASIAN, then we get the mean.
# y <- auck$PC_ASIAN
# ybar <- mean(y)

# 3. find the difference between y and ybar(the mean)
# dy <- y - ybar
# yi <- rep(dy, each=n)
# yj <- rep(dy)
# yiyj <- yi * yj

# pm <- matrix(yiyj, ncol=n)

# pmw <- pm * wm

# spmw <- sum(pmw)

# smw <- sum(wm)
# sw  <- spmw / smw
# vr <- n / sum(dy^2)
# MI <- vr * sw

# 4. Morans I
# cat("Moran's I is", MI)

In [ ]:
# let's make a weights object so that we 
# can use it with a less manual way of computing moran's i
lw <- nb2listw(nb, style="W", zero.policy=TRUE)
print(lw, zero.policy=TRUE)

In [ ]:
summary(lw, zero.policy=TRUE)

In [ ]:

m_scatterplot <- function (sf, varname, listweights, ttl='', linecol='red') {
    
scaled_col <<- paste('s', varname, sep='')
lagged_col <<- paste('lag_', varname, sep='')
print(scaled_col)
print(lagged_col)

sf[[scaled_col]] <- scale(sf[[varname]]) 
    
sf[[lagged_col]] <- lag.listw(listweights, sf[[scaled_col]], zero.policy=TRUE)
    
plot(x=sf[[scaled_col]], y=sf[[lagged_col]], xlab=scaled_col, ylab=lagged_col, main=ttl)
    
abline(h=0, v=0)
best_fit_line <- lm(sf[[lagged_col]] ~ sf[[scaled_col]])
abline(best_fit_line, lty=2, lwd=1, col=linecol)
    
#Note that the slope of the regression line 
# is nearly the same as Moran's I
coefficients(best_fit_line)[2]
    
# Save new columns into the shapefile
assign('auck',sf, envir=.GlobalEnv)
}

In [ ]:
#Call the function to make the plot
m_scatterplot(auck, 'ASI_P_06', lw, ttl='Moran Scatterplot Percent Asian', linecol='red')

#### How did we change our shapefile?

Let's look at the table format again, with the head() function

In [ ]:
# head(auck@data, n=10)

In [ ]:
moran(auck$ASI_P_06, lw, n=length(lw$neighbours), S0=Szero(lw), NAOK=TRUE, zero.policy=TRUE)

In [ ]:
moran.test(auck$ASI_P_06, lw, randomisation=FALSE, zero.policy=TRUE)

In [ ]:
mmc <- moran.mc(auck$ASI_P_06, lw, nsim=999, zero.policy=TRUE)

In [ ]:
hist(mmc$res, main="Histogram of results from permutation", xlab="Moran's index")
abline(v=mmc$statistic, col='red', lty=2)

# Univariate Local Moran’s I

Local Moran is defined as:

$I_i = \frac{(x_i-\bar{x})}{{\sum_{k=1}^{n}(x_k-\bar{x})^2}/(n-1)}{\sum_{j=1}^{n}w_{ij}(x_j-\bar{x})}$

In [ ]:
# We use the localmoran function instead of moran
# let's title our results "locm_asi_p_o6" for clarity
locm_asi_p_06 <- localmoran(auck$ASI_P_06, lw, alternative="two.sided")
summary(locm_asi_p_06, NAOK = TRUE)

### What exactly do the above numbers mean?


In [ ]:
# Let's print out the locm dataframe, which is what locm (local moran)
# gives back to us
number_of_rows <- nrow(locm_asi_p_06) 

How many number of rows are there? 
How does this compare to the number of rows in the shapefile?

What's the output of locm look like? 
We'll just look at the first 10 rows to get a quick sense of the data.


In [ ]:
head(locm_asi_p_06, n=10)

**Fortunately** there is some very nice documentation on locm, especially on the columns it outputs.

It's here: https://www.rdocumentation.org/packages/spdep/versions/0.7-4/topics/localmoran

`Ii` is the `local moran statistic`

`E.Ii` is the `expectation of local moran statistic`

`Var.Ii` is the `variance of local moran statistic`

`Z.Ii` is the `standard deviate of local moran statistic`

`Pr()` is the `p-value of local moran statistic`

### LISA Cluster map

From your readings about local moran, we'll want to make some quadrants from the scatterplot, in order to make maps that indicate clustering.

These quadrants will go into Local Indicators of Spatial Association (LISA) Cluster maps.

Local Indicators of Spatial Association (LISA) tests for regional clustering and the presence of significant spatial clusters or outliers.

The LISA Significance Map shows significant results by tract. 

The LISA Cluster Map shows how the attributes cluster.  In the example below using the specific color gradient, the red color shows tracts where high rate cluster with high rates, and blue shows where low rates cluster with low rates.  

- High-high and low-low = spatial clusters
- High-low and low-high = spatial outliers

We need to make some quadrants from the data for our map. First, let's think back to the scatterplot.

If you think back to the scatterplot, there's essentially 4 quadrants in the scatterplot.

|- | + |
|---|---|
|  4  |  1  |
|  2   | 3   |

scaled_col is the x-axis
lagged_col is the y-axis

- 1 is where scaled_col is greater than 0, lagged_col is greater than 0, high-high
- 2 is where scaled_col is less than 0, lagged_col is less than 0, low-low
- 3 is where scaled_col is greater than zero and lagged_col is less than 0, high-low
- 4 is where scaled_col is less than 0, lagged_col is greater than 0, low-high

In [ ]:
# Identify the moran plot quadrant for each 
# observation to make the cluster map

lisa_quadrant_cluster <- function (sf, locm_table, data_col, scaled_col, lagged_col, sig=0.05) {
    # First, let's make an empty QUAD_SIG data column for the data we're looking at, in the shapefile
    quad_sig_col <<- paste('QUAD_SIG_', data_col, sep='')
    
    sf[[quad_sig_col]] <- NA
    
    # Then let's assign quadrants
    sf[[quad_sig_col]][(sf[[scaled_col]] >= 0 & sf[[lagged_col]] >= 0) & (locm_table[, 5] <= sig)] <- 1
    sf[[quad_sig_col]][(sf[[scaled_col]] <= 0 & sf[[lagged_col]] <= 0) & (locm_table[, 5] <= sig)] <- 2
    sf[[quad_sig_col]][(sf[[scaled_col]] >= 0 & sf[[lagged_col]] <= 0) & (locm_table[, 5] <= sig)] <- 3
    sf[[quad_sig_col]][(sf[[scaled_col]] <= 0 & sf[[lagged_col]] >= 0) & (locm_table[, 5] <= sig)] <- 4
    sf[[quad_sig_col]][(sf[[scaled_col]] <= 0 & sf[[lagged_col]] >= 0) & (locm_table[, 5] <= sig)] <- 5
    #5 are non significant observations
    assign('auck',sf, envir=.GlobalEnv)
}


In [ ]:
lisa_quadrant_cluster(auck, locm_asi_p_06, 'ASI_P_06','sASI_P_06', 'lag_ASI_P_06', sig=0.05)

#head(auck@data,n=10)

In [ ]:
# The function to make our first LISA cluster map
local_moran_cluster_map <- function (sf, 
                                     quad_sig_column, 
                                     breaks=c(1,2,3,4,5), 
                                     labels, colors, ttl="Local Moran's I") {
    
    # Set the corresponding labels for the thematic map classes
    numberOfIntervals <- findInterval(sf[[quad_sig_column]], breaks)

    # Generate the map
    plot(auck, col = pal[numberOfIntervals])
    mtext(ttl, cex = 1.5, side =3, line = 1)
    legend("topleft", legend = labels, fill = pal, bty = "n")

}


In [ ]:

# Set the labeling for the map legend. These are the high-high, low-low etc 
# classes we calculated from the quadrants ealier.
labeling <- c("High-High", "Low-Low", "High-Low", "Low-High", "Not Signif.")

# Set the breaks for the thematic map classes
# We use the seq function to generation a sequence from 1-5, going up by 1.
num_breaks <- 1:6

# Define color swatches
pal <- c("red", "blue", "lightpink", "skyblue2", "white")


local_moran_cluster_map(auck, 'QUAD_SIG_ASI_P_06', num_breaks, labeling, colors, ttl="Local Moran's I")

### LISA Significance map

In [ ]:
# The significance map made from the p values that localmoran() gave us.
# Reminder that the p values (significance) is column number 5 in locm
# We'll want to copy over the p-values from the locm results to our shapefile first
# So that we can map the values by color


local_moran_sig_map <- function (sf, 
                                 locm_table, 
                                 new_column_name, 
                                 plabels, breakpoints, num_breaks=5, 
                                 ttl="LISA Significance Map",
                                 palcolors="Greens") {
    # Make a column in auck data that has NA
    sf[[new_column_name]] <- NA
    
    # Populate LOCM_P with the results from locm function
    sf[[new_column_name]] <- locm_table[,5]
    sf[[new_column_name]]
    classes <- classIntervals(sf[[new_column_name]], num_breaks, style="fixed", fixedBreaks=breakpoints)
    
    #How many items do we have in each of our class intervals?
    print(classes)
    
    palette <- rev(brewer.pal(num_breaks, palcolors))
    colors <- findColours(classes, palette)

    # Generate the map
    plot(auck, col = colors, lwd=0.2, main=ttl)

    legend("topleft", legend = plabels, fill = palette, bty = "n")
    assign('auck',sf, envir=.GlobalEnv)

}


In [ ]:
pval_labels <- c("p<=0.0001", "p<=0.001","p<=0.01", "p<=0.05","Not Significant")
pval_fixed_breaks <- c(1,0.05,0.01,0.001,0.0001,0)

#head(auck@data, n=10)
local_moran_sig_map(auck,
                    locm_asi_p_06,
                    'LOCM_ASI_P_06', 
                    pval_labels, 
                    pval_fixed_breaks, 
                    5, ttl="LISA Significance Map", "Greens")

# Questions + Deliverables

1. Determine the Moran’s I statistic for:
    - TB Rates 
    - Maori census groupings
    - Asian census groupings
    - Pacific Islander groupings
    - European groupings. 
  
2. For one (you choose) of the four major groupings perform the Univariate Local Moran’s
analysis. You should produce a Moran scatter plot, Significance map and Cluster map for this
analysis, and also a standard map (choose the map type you consider most informative)

3. Write a short report addressing the questions below:

    **Q1**:

    Compile a table of the Univariate Moran’s I results for each of the four major census groups. Which is most ‘aggregated’ based on these results? Do you think this result is very meaningful? Explain your answer with respect only to the statistical results, not the geographical distributions. 

    **Q2**:

    What correlation are there between the TB Rates and the census groupings? Are the TB Rates geographically clustered? 

    **Q3**:

    What changes to the Moran’s I approach might identify the difference among the groups more effectively? 
    [Hint: This is not an easy question, and you are not expected to come up with a definitive answer. Think about (among other things): scale, the census polygons being used, the total populations of each group, and how we are considering ‘near’ when we use polygon contiguity.